In [0]:
%sql

CREATE CONNECTION IF NOT EXISTS earthquake_conn
TYPE HTTP
OPTIONS (
  host 'https://earthquake.usgs.gov',
  port '443',
  base_path '/earthquakes/feed/v1.0/',
  bearer_token 'na'
)



In [0]:
dbutils.widgets.text('schema_name','bronze','Schema Name')
dbutils.widgets.text('catalog_name','dev01','Catalog Name')
schema = dbutils.widgets.get('schema_name')
catalog = dbutils.widgets.get('catalog_name')



In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
display([c.as_dict() for c in w.connections.list()])

In [0]:
conn = w.connections.get(name="earthquake_conn").as_dict()

host = conn["options"]["host"]
base_path = conn["options"]["base_path"]


In [0]:
from datetime import datetime, timezone

current_datetime = datetime.now().strftime("%Y-%m-%d")




In [0]:
import requests
import json

endpoint = '/summary/all_day.geojson'
url = f"{host}{base_path}{endpoint}"

url = f"{host}{base_path}{endpoint}"


try:
    response = requests.get(url)
except Exception as e:
    print(f"Error: {e}")
if response.status_code != 200:
    raise Exception(f"Error: {response.status_code} - {response.text}")
elif response.status_code == 200:
    print(f"Success: {response.status_code} - {response.text}")


response.json()

In [0]:

spark.sql(f'use catalog {catalog}')
spark.sql(f'create schema if not exists {schema}')
spark.sql(f'use {schema}')
spark.sql('create volume if not exists earthquake_volume')

In [0]:
import os

data = response.json()

file_path = f'/Volumes/{catalog}/{schema}/earthquake_volume/all_day_{current_datetime}.json'
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f)
    print(f"File saved to {file_path}")
else:
    print(f"File already exists at {file_path}")